In [4]:
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
from ctapipe.utils import get_dataset
from ctapipe.image import tailcuts_clean
from ctapipe.reco import HillasReconstructor
from ctapipe.image import hillas_parameters
from ctapipe.reco.HillasReconstructor import TooFewTelescopesException
import hipecta.memory
import hipecta.calib
import astropy.units as u
import numpy as np
import copy

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
filename = get_dataset("gamma_test.simtel.gz")

In [7]:
events = [copy.deepcopy(event) for event in event_source(filename)]

In [8]:
cal = CameraCalibrator(None, None, r1_product='HESSIOR1Calibrator', extractor_product='NeighbourPeakIntegrator')

In [9]:
hillas_reco = HillasReconstructor()

In [10]:
class HillasNotFinite(Exception):
    """
    Error to be raised when hillas parameters are not finite
    """
    pass

In [11]:
def reconstruction(event, picture_thresh=6, boundary_thresh=0):
    
    cal.calibrate(event)
    
    features = {}
    hillas_dict = {}
    pointing_azimuth = {}
    pointing_altitude = {}
    for telescope_id, dl1 in event.dl1.tel.items():

        camera = event.inst.subarray.tels[telescope_id].camera
        mask = tailcuts_clean(camera,
                              dl1.image[0], 
                              boundary_thresh=boundary_thresh, 
                              picture_thresh=picture_thresh)
        
        telescope_type_name = event.inst.subarray.tels[telescope_id].optics.tel_type
        image = dl1.image[0]
        image[~mask] = 0

        if image.sum() > 0:
            try:
                h = hillas_parameters(
                    camera,
                    image
                )

                if not all(map(np.isfinite, h.values())):
                    raise HillasNotFinite("bad Hillas parameters")
                    
                hillas_dict[telescope_id] = h
                pointing_azimuth[telescope_id] = event.mc.tel[telescope_id].azimuth_raw * u.rad
                pointing_altitude[telescope_id] = event.mc.tel[telescope_id].altitude_raw * u.rad
                # pointing_altitude[telescope_id] = ((np.pi/2) - event.mc.tel[telescope_id].altitude_raw )* u.rad # this is weird to say the least. 
    
            except HillasNotFinite:
                pass
                
        else:
            print("image sum < 0", image.sum())

    if len(hillas_dict) < 2:
        print("mono")
        # raise TooFewTelescopesException()
        reco = None
    else:
        reco = hillas_reco.predict(hillas_dict, event.inst, pointing_azimuth, pointing_altitude)
   
    return reco


In [12]:
def process_events(filename):
    events = [copy.deepcopy(event) for event in event_source(filename)]
    print(len(events))
    reco = []
    for i, event in enumerate(events):
        print(i)
        reco.append(reconstruction(event))
    return reco

In [13]:
reco = process_events(filename);

9
0
1
2
3
4
5
6
7
8
image sum < 0 0.0


In [15]:
for r, event in zip(reco, events):
    print(event.r0.event_id)
    print(event.mc.alt, event.mc.az)
    print(r.alt, r.az)
    print('-----')

408
1.22173rad 6.28319rad
0.00710163rad 4.34482rad
-----
409
1.22173rad 6.28319rad
0.0260324rad 4.3533rad
-----
803
1.22173rad 6.28319rad
0.00161457rad 4.36529rad
-----
4907
1.22173rad 6.28319rad
0.0193678rad 1.22404rad
-----
9508
1.22173rad 6.28319rad
0.0172549rad 1.20392rad
-----
10104
1.22173rad 6.28319rad
0.00232797rad 1.20654rad
-----
10109
1.22173rad 6.28319rad
0.00397331rad 4.35761rad
-----
11905
1.22173rad 6.28319rad
0.00123515rad 1.2216rad
-----
12202
1.22173rad 6.28319rad
0.0211464rad 1.24256rad
-----
